In [1]:
!pip install transformers datasets sacrebleu sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [1]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
# !rm -rf ~/.cache/huggingface/datasets/MBZUAI-Paris_DODa-10K
# !git clone https://huggingface.co/datasets/MBZUAI-Paris/DODa-10K
!git clone https://huggingface.co/datasets/midox05/train_ready_english_darija_dataset

Cloning into 'train_ready_english_darija_dataset'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7 (from 1)
Unpacking objects: 100% (7/7), 2.25 KiB | 2.25 MiB/s, done.


In [18]:
from datasets import load_dataset, Dataset
import pandas as pd
import random

# dataset = load_dataset('midox05/train_ready_english_darija_dataset', split='train')
dataset = pd.read_csv("/content/train_ready_english_darija_dataset/ready_df.csv")


In [19]:
dataset = Dataset.from_pandas(dataset)

# Obtenez 60000 indices aléatoires
random_indices = random.sample(range(len(dataset)), 40000)

# Sélectionnez les lignes correspondantes
random_dataset = dataset.select(random_indices)

dataset = random_dataset.train_test_split(test_size=0.1)

In [20]:
def preprocess(examples):
    inputs = examples["english"]
    targets = examples["darija"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [21]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt_en_ar_finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-21-f4303e7fcb12>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,0.352200,0.295405,34.500840
2,0.282400,0.277523,36.220532
3,0.199400,0.271230,37.027775


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=13500, training_loss=0.28971951738110296, metrics={'train_runtime': 4749.3985, 'train_samples_per_second': 22.74, 'train_steps_per_second': 2.842, 'total_flos': 3661023412224000.0, 'train_loss': 0.28971951738110296, 'epoch': 3.0})

In [23]:
from datasets import load_metric
import numpy as np

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    return {"bleu": result["score"]}


In [24]:
import torch

def translate(texts):
    # Tokenize the input text
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # Generate translations using the model
    with torch.no_grad():
        translated = model.generate(**inputs, max_length=128)

    # Decode the output tokens to get the translated sentences
    return tokenizer.batch_decode(translated, skip_special_tokens=True)


In [25]:
# Example English sentences
sentences = [
    "The weather is nice today.",
    "Could you please pass the salt?",
    "I am learning a new language.",
    "What time is it?",
    "This is a beautiful day for a walk."
]

# Get Arabic translations
translations = translate(sentences)

# Show results
for en, ar in zip(sentences, translations):
    print(f"EN: {en}\nAR: {ar}\n")


EN: The weather is nice today.
AR: الطقس زوين اليوم.

EN: Could you please pass the salt?
AR: واش ممكن تعطيني الملح؟

EN: I am learning a new language.
AR: أنا كنتعلم لغة جديدة.

EN: What time is it?
AR: شحال من وقت كاين؟

EN: This is a beautiful day for a walk.
AR: هادي نهار زوين باش تمشي.



In [26]:
model.save_pretrained("darija-eng-trsl")
tokenizer.save_pretrained("darija-eng-trsl")

('darija-eng-trsl/tokenizer_config.json',
 'darija-eng-trsl/special_tokens_map.json',
 'darija-eng-trsl/vocab.json',
 'darija-eng-trsl/source.spm',
 'darija-eng-trsl/target.spm',
 'darija-eng-trsl/added_tokens.json')

In [28]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `ProjetDl` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ProjetDl`

In [29]:
# Replace with your model repo name
model_name_on_hub = "nwiser/darija-eng-trsl"

# Push model
model.push_to_hub(model_name_on_hub)

# Push tokenizer
tokenizer.push_to_hub(model_name_on_hub)

model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nwiser/darija-eng-trsl/commit/1a458d916f43868075aeeec8fd04e906272226e1', commit_message='Upload tokenizer', commit_description='', oid='1a458d916f43868075aeeec8fd04e906272226e1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nwiser/darija-eng-trsl', endpoint='https://huggingface.co', repo_type='model', repo_id='nwiser/darija-eng-trsl'), pr_revision=None, pr_num=None)